# InSAR denoiser training, validation

In [1]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import torch
import glob
from sklearn.model_selection import train_test_split
from PIL import Image
import seaborn as sns
import math
from skimage.metrics import structural_similarity as ssim
import random
from scipy import stats

## Dataset construction

In [2]:
# set paths
main_dir = '/home/jovyan/InSAR_denoising_CNN'
train_signal_dir = f'{main_dir}/train_subsets_v3/veloc/'
train_noise_dir = f'{main_dir}/train_subsets_v3/int/'
train_dem_dir = f'{main_dir}/train_subsets_v3/dem/'
train_era5_dir = f'{main_dir}/train_subsets_v3/era5/'

val_signal_dir = f'{main_dir}/val_subsets_v3/veloc/'
val_noise_dir = f'{main_dir}/val_subsets_v3/int/'
val_dem_dir = f'{main_dir}/val_subsets_v3/dem/'
val_era5_dir = f'{main_dir}/val_subsets_v3/era5/'

# list files
train_signal_fns = os.listdir(train_signal_dir)
train_noise_fns = os.listdir(train_noise_dir)
train_dem_fns = os.listdir(train_dem_dir)
train_era5_fns = os.listdir(train_era5_dir)

val_signal_fns = os.listdir(val_signal_dir)
val_noise_fns = os.listdir(val_noise_dir)
val_dem_fns = os.listdir(val_dem_dir)
val_era5_fns = os.listdir(val_era5_dir)

# exclude non tif files, e.g. metadata
def list_tifs(my_fns):
    my_list = []
    for i in my_fns:
        if i[-4:] == '.tif':
            my_list.append(i)
    return my_list

train_signal_list = list_tifs(train_signal_fns)
train_noise_list = list_tifs(train_noise_fns)
train_dem_list = list_tifs(train_dem_fns)
train_era5_list = list_tifs(train_era5_fns)

val_signal_list = list_tifs(val_signal_fns)
val_noise_list = list_tifs(val_noise_fns)
val_dem_list = list_tifs(val_dem_fns)
val_era5_list = list_tifs(val_era5_fns)

# create training list of only scenes shared in all necessary dirs
train_list = []
for fn in train_signal_list:
    if fn in train_noise_list and fn in train_dem_list and fn in train_era5_list:
        train_list.append(fn)
        
val_list = []
for fn in val_signal_list:
    if fn in val_noise_list and fn in val_dem_list and fn in val_era5_list:
        val_list.append(fn)

In [ ]:
# split training and validation data 
#train_list, val_list = train_test_split(train_list, test_size=0.2)

In [3]:
# define transforms
my_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5)
])

In [4]:
# define dataset 
class dataset(torch.utils.data.Dataset):
    def __init__(self, file_list, signal_dir, noise_dir, dem_dir, era5_dir, transform=None, 
                 norm=True, center=True, blurnoise=False):
        self.file_list = file_list
        self.transform = transform
        self.signal_dir = signal_dir
        self.noise_dir = noise_dir
        self.dem_dir = dem_dir
        self.era5_dir = era5_dir
        self.norm = norm
        self.center = center
        self.blurnoise = blurnoise
        
    #dataset length
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    #load images
    def __getitem__(self,idx):
        signal_path = self.signal_dir+self.file_list[idx]
        noise_path = self.noise_dir+self.file_list[idx]
        dem_path = self.dem_dir+self.file_list[idx]
        era5_path = self.era5_dir+self.file_list[idx]
        
        signal = self.transform(Image.open(signal_path))
        noise = self.transform(Image.open(noise_path))
        dem = self.transform(Image.open(dem_path))
        era5 = self.transform(Image.open(era5_path))
        
        # Generate era5 noise estimates
        era5 = era5*(0.05546576/12.5663706) # convert phase to displacement
        era5 = (noise-(era5*-1)) 
        
        # Blur noise
        if self.blurnoise == True: # blur noise to mitigate noise from non atmospheric sources
            gblur = transforms.GaussianBlur(kernel_size=(7, 7), sigma=5)
            noise = gblur(noise)
        
        # Generate scaled training images
        scalar = np.round(np.random.lognormal(0.05, 1.), 3)
        signal = signal*scalar
        train = noise+signal
        
        # Set era reference point
        ref_index = signal.abs().argmin().item() # location of lowest signal in velocity map
        corr_diff = (train.flatten()[ref_index] - era5.flatten()[ref_index]).item()
        era5 = era5+corr_diff 
        
        # correct train
        era5_corr = train-era5 #produce era5 corrected train image
        
        # correct hp
        hp_filter = transforms.GaussianBlur(kernel_size=(25, 25), sigma=3)
        train_filtered = hp_filter(train)
        hp_corr = train - train_filtered
        
        # normalization between -1 and 1 as in Zhao et al. https://doi.org/10.1016/j.isprsjprs.2021.08.009
        if self.norm == True:
            if train.min() < signal.min():
                norm_min = train.min()
            else:
                norm_min = signal.min()
                
            if train.max() > signal.max():
                norm_max = train.max()
            else:
                norm_max = signal.max()
            
            signal = 2*(((signal-norm_min)/(norm_max-norm_min)))-1
            noise  = 2*(((noise-(noise.min()))/(noise.max()-(noise.min()))))-1
            dem = 2*(((dem-dem.min())/(dem.max()-dem.min())))-1
            train = 2*(((train-norm_min)/(norm_max-norm_min)))-1
            era5 = 2*(((era5-era5.min())/(era5.max()-era5.min())))-1
            era5_corr = 2*(((era5_corr-norm_min)/(norm_max-norm_min)))-1
            hp_corr = 2*(((hp_corr-norm_min)/(norm_max-norm_min)))-1
        
        if self.center == True: # center target images on 0 
            center_median = signal.median()
            train = train-center_median
            signal = signal-center_median
            era5_corr = era5_corr-center_median
            hp_corr = hp_corr-center_median
         
        
        return train, signal, noise, era5, dem, era5_corr, hp_corr

In [5]:
# create dataloaders
train_data = dataset(train_list, train_signal_dir, train_noise_dir, train_dem_dir, train_era5_dir, transform=my_transforms, blurnoise=True)
val_data = dataset(val_list, val_signal_dir, val_noise_dir, val_dem_dir, val_era5_dir, transform=my_transforms, blurnoise=True)

train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=1, shuffle=True)

## Examine dataset

In [ ]:
num_images = 5

for i, (sample, signal_target, noise_target, era5_noise, dem, era5_corr, hp_corr) in enumerate(val_loader):
    if i < num_images:
            f, ax = plt.subplots(1, 5, figsize=(20,7))
            ax[0].imshow(sample.squeeze(), cmap='RdBu', vmin=-2, vmax=2) 
            ax[0].set_title('training')
            ax[1].imshow(signal_target.squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[1].set_title('target signal')
            ax[2].imshow((sample.squeeze()-signal_target.squeeze()), cmap='RdBu', vmin=-2, vmax=2)
            ax[2].set_title('target noise')
            ax[3].imshow(era5_noise.squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[3].set_title('ERA5 noise')
            ax[4].imshow(dem.squeeze(), cmap='gray', vmin=-2, vmax=2)
            ax[4].set_title('DEM')
            f.tight_layout()
            
            f, ax = plt.subplots(1, 5, figsize=(15,3))
            ax[0].hist(sum(sum(sum(sample.tolist(), []), []), []), bins=40)
            ax[1].hist(sum(sum(sum(signal_target.tolist(), []), []), []), bins=40)
            ax[2].hist(sum(sum(sum((sample-signal_target).tolist(), []), []), []), bins=40)
            ax[3].hist(sum(sum(sum(era5_noise.tolist(), []), []), []), bins=40)
            ax[4].hist(sum(sum(sum(dem.tolist(), []), []), []), bins=40)
            f.tight_layout()

## Define network

In [7]:
def conv3x3(in_channels, out_channels, stride=1, padding=1, bias=True):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=padding, bias=bias)


def conv1x1(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)


def check_valid_activation(choice):
    if choice not in ['relu', 'lrelu', 'prelu']:
        raise ValueError(f"'{choice}' is not a valid activation function. Choose among ['relu', 'lrelu', 'prelu'].\n")


def upconv(in_channels, out_channels, mode='transpose'):
    # stride=2 implies upsampling by a factor of 2
    get_up_mode = nn.ModuleDict([
        ['bilinear', nn.Sequential(nn.Upsample(mode='bilinear', scale_factor=2), conv1x1(in_channels, out_channels))],
        ['transpose', nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)]
    ])

    return get_up_mode[mode]


def get_activation(choice):
    activation_functions = nn.ModuleDict([
        ['relu', nn.ReLU(inplace=True)],
        ['lrelu', nn.LeakyReLU(inplace=True)],
        ['prelu', nn.PReLU()]
        ])
    return activation_functions[choice]


def conv_block(in_channels, out_channels, activation='relu', do_BN=True, *args, **kwargs):
    """
    Partial encoder block consisting of a 3×3 convolutional layer with stride 1, followed by batch normalization
    (optional) and a non-linear activation function.
    """

    if do_BN:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=False, *args, **kwargs),
            nn.BatchNorm2d(out_channels),
            get_activation(activation)
        )
    else:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=True, *args, **kwargs),
            get_activation(activation)
        )


def conv_up_block(in_channels, out_channels, activation='relu', do_BN=True, up_mode='transpose', *args, **kwargs):
    """
    Decoder block consisting of an up-convolutional layer, followed by a 3×3 convolutional layer with stride 1,
    batch normalization (optional), and a non-linear activation function.
    """

    if do_BN:
        return nn.Sequential(
            upconv(in_channels, in_channels, up_mode),
            nn.Sequential(
                conv3x3(in_channels, out_channels, bias=False, *args, **kwargs),
                nn.BatchNorm2d(out_channels),
                get_activation(activation))
            )
    else:
        return nn.Sequential(
            upconv(in_channels, in_channels, up_mode),
            nn.Sequential(
                conv3x3(in_channels, out_channels, bias=True, *args, **kwargs),
                get_activation(activation))
            )


def bottleneck(in_channels, out_channels, activation='relu', do_BN=True, *args, **kwargs):
    """
    Bottleneck block.
    """

    if do_BN:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=False, *args, **kwargs),
            nn.BatchNorm2d(out_channels),
            get_activation(activation)
        )
    else:
        return nn.Sequential(
            conv3x3(in_channels, out_channels, bias=True, *args, **kwargs),
            get_activation(activation)
        )


class SkipConnection(nn.Module):
    def __init__(self):
        super(SkipConnection, self).__init__()

    def forward(self, x_skip, x_up):
        return x_skip + x_up


class UNet(nn.Module):
    def __init__(self, n_input_channels=2, start_kernel=64, max_filter_depth=512, depth=7,
                 act_fn_encoder='relu', act_fn_decoder='relu', act_fn_bottleneck='relu', up_mode='transpose',
                 do_BN=True, bias_conv_layer=False, outer_skip=True, outer_skip_BN=False):
        """
        UNet network architecture.
        :param n_input_channels:    int, number of input channels
        :param start_kernel:        int, number of filters of the first convolutional layer in the encoder
        :param max_filter_depth:    int, maximum filter depth
        :param depth:               int, number of downsampling and upsampling layers (i.e., number of blocks in the
                                    encoder and decoder)
        :param act_fn_encoder:      str, activation function used in the encoder
        :param act_fn_decoder:      str, activation function used in the decoder
        :param act_fn_bottleneck:   str, activation function used in the bottleneck
        :param up_mode:             str, upsampling mode
        :param do_BN:               boolean, True to perform batch normalization after every convolutional layer,
                                    False otherwise
        :param bias_conv_layer:     boolean, True to activate the learnable bias of the convolutional layers,
                                    False otherwise
        :param outer_skip:          boolean, True to activate the long residual skip connection that adds the
                                    initial DSM to the output of the last decoder layer, False otherwise
        :param outer_skip_BN:       boolean, True to add batch normalization to the long residual skip connection,
                                    False otherwise
        """

        super(UNet, self).__init__()

        check_valid_activation(act_fn_encoder)
        check_valid_activation(act_fn_decoder)
        check_valid_activation(act_fn_bottleneck)

        if up_mode not in ['transpose', 'bilinear']:
            raise ValueError(f"'{up_mode}' is not a valid mode for upsampling. Choose among ['transpose', 'bilinear'] "
                             "to specify 'up_mode'.\n")

        self.n_input_channels = n_input_channels
        self.start_kernel = start_kernel
        self.depth = depth
        self.act_fn_encoder = act_fn_encoder
        self.act_fn_decoder = act_fn_decoder
        self.act_fn_bottleneck = act_fn_bottleneck
        self.up_mode = up_mode
        self.max_filter_depth = max_filter_depth
        self.do_BN = do_BN
        self.bias_conv_layer = bias_conv_layer
        self.do_outer_skip = outer_skip
        self.do_outer_skip_BN = outer_skip_BN
        self.filter_depths = [self.start_kernel * (2 ** i) for i in range(self.depth)]

        # Restrict the maximum filter depth to a predefined value
        self.filter_depths = [self.max_filter_depth if i > self.max_filter_depth else i for i in self.filter_depths]

        # Set up the encoder
        self.encoder = nn.ModuleList()
        self.encoder.append(nn.Sequential(
            conv_block(self.n_input_channels, self.start_kernel, activation=self.act_fn_encoder, do_BN=self.do_BN),
            nn.MaxPool2d(kernel_size=2, stride=2)
            ))

        for in_channel, out_channel in zip(self.filter_depths, self.filter_depths[1:]):
            self.encoder.append(nn.Sequential(
                conv_block(in_channel, out_channel, activation=self.act_fn_encoder, do_BN=self.do_BN),
                nn.MaxPool2d(kernel_size=2, stride=2)
            ))

        # Set up the bottleneck
        self.bottleneck = bottleneck(self.filter_depths[-1], self.filter_depths[-1], activation=self.act_fn_bottleneck,
                                     do_BN=self.do_BN)

        # Set up the decoder
        self.decoder = nn.ModuleList()
        self.filter_depths_up = list(reversed(self.filter_depths))

        for in_channel, out_channel in zip(self.filter_depths_up[:-1], self.filter_depths_up[1:]):
            self.decoder.append(conv_up_block(in_channel, out_channel, activation=self.act_fn_decoder,
                                              up_mode=self.up_mode, do_BN=self.do_BN))
        self.decoder.append(upconv(self.filter_depths_up[-1], self.filter_depths_up[-1], up_mode))

        # Set up the final layer of the decoder
        self.last_layer = conv3x3(self.start_kernel, 1, bias=self.bias_conv_layer)

        # Skip connection
        self.skipconnect = SkipConnection()

        # Batch normalization added to the long residual skip connection
        if self.do_outer_skip:
            self.layer_outer_skip = nn.ModuleList()
            if self.do_outer_skip_BN:
                self.layer_outer_skip.append(nn.BatchNorm2d(1))
            self.layer_outer_skip.append(SkipConnection())

    def forward(self, x, dem):
        skip_connections = []
        x = torch.cat((x, dem), dim=1)
        out = x

        # Encoder (save intermediate outputs for skip connections)
        for index, layer in enumerate(self.encoder):
            layer_conv = layer[:-1]  # all layers before the pooling layer (at depth index)
            layer_pool = layer[-1]   # pooling layer (at depth index)

            out_before_pool = layer_conv(out)
            skip_connections.append(out_before_pool)
            out = layer_pool(out_before_pool)

        # Bottleneck
        out = self.bottleneck(out)

        # Decoder + skip connections
        index_max = len(self.decoder) - 1
        for index, layer in enumerate(self.decoder):
            if index <= index_max - 1:
                layer_upconv = layer[0]  # upconv layer
                layer_conv = layer[1::]  # all other layers (conv, batchnorm, activation)

                out_temp = layer_upconv(out)
                out = self.skipconnect(skip_connections[-1 - index], out_temp)
                out = layer_conv(out)
            else:
                out_temp = layer(out)   # upconv of last layer
                out = self.skipconnect(skip_connections[-1 - index], out_temp)

        # Last layer of the decoder
        out = self.last_layer(out)

        # Add long residual skip connection
        if self.do_outer_skip:
            if self.layer_outer_skip.__len__() == 2:
                # pipe input through a batch normalization layer before adding it to the output of the last
                # decoder layer
                bn = self.layer_outer_skip[0]
                x_0 = x[:, 0, :, :]       # use channel 0 only
                x_0 = x_0.unsqueeze(1)
                x = bn(x_0)

            # add (batchnorm) input to the output of the last decoder layer
            add = self.layer_outer_skip[-1]
            x_0 = x[:, 0, :, :]
            x_0 = x_0.unsqueeze(1)

            out = add(x_0, out)  # use channel 0 only

        return out

## Train model

In [ ]:
#load previous model
model = UNet()
model.load_state_dict(torch.load('noisemodelv4.2_20epochs'))
model.to('cuda')
model.eval()

In [ ]:
%%time

#Define optimizer
model = UNet()
model.to('cuda') # run on gpu
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0002, weight_decay=0.02) # set learning rate
loss_fn   = nn.L1Loss() # MAE loss function, doesn't penalize outliers as much as MSE
epochs = 50
quantile = 0.9 # full loss computed for pixels with magnitude above this quantile 
stable_scalar = 1 # scalar for devaluing pixels with magnitude below quantile in loss calc
torch.autograd.set_detect_anomaly(True)

train_loss = []
val_loss = []

for epoch in range(epochs):
    print(f'\nstarting epoch {epoch}')
    epoch_loss=[]
    val_temp_loss = []
    
    if epoch == 10:
        optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001) # reduce loss as given epoch
    
    #loop through training data 
    for (sample, signal_target, noise_target, era5_noise, dem, era5_corr, hp_corr) in train_loader:
        model.train()
        optimizer.zero_grad()
        out = torch.clamp(model(sample.to('cuda'), dem.to('cuda')), -1, 1) # Generate noise predictions
        
        # isolate high-signal areas for loss calculation
        # find quantile values for given batch of images
        q = signal_target.to('cuda').abs().flatten(1, 3).quantile(dim=1, q=quantile, keepdim=True) 
        # scale down target values in pixels with magnitude below quantile for loss calc
        signal_target_high = torch.where(signal_target.to('cuda').abs()>q.to('cuda')[:, None, None], 
                                         signal_target.to('cuda'), signal_target.to('cuda')*stable_scalar)
        # calculate predicted signals 
        signal_pred = sample.to('cuda')-out.to('cuda')
        # scale down predicted signal values in pixels with magnitude below quantile for loss calc
        signal_pred_high = torch.where(signal_target.to('cuda').abs()>q.to('cuda')[:, None, None], 
                                       signal_pred.to('cuda'), signal_pred.to('cuda')*stable_scalar)
        
        loss = loss_fn(signal_pred_high.to('cuda'), signal_target_high.to('cuda')) # calculate loss 
        epoch_loss.append(loss.item()) # add batch loss to epoch loss list
        
        loss.backward() #Propagate the gradients in backward pass
        optimizer.step() 

    train_loss.append(np.mean(epoch_loss))
    print(f'training loss: {np.mean(epoch_loss)}')
    
    # run model on validation data 
    for (sample, signal_target, noise_target, era5_noise, dem, era5_corr, hp_corr) in val_loader:
        with torch.no_grad():
            model.eval()
            out = torch.clamp(model(sample.to('cuda'), dem.to('cuda')), -1, 1) #Generate predictions using the model
            
            # isolate high-signal areas for loss calculation
            q = signal_target.to('cuda').abs().flatten(1, 3).quantile(dim=1, q=quantile, keepdim=True)
            signal_target_high = torch.where(signal_target.to('cuda').abs()>q.to('cuda')[:, None, None], 
                                         signal_target.to('cuda'), signal_target.to('cuda')*stable_scalar)   
            signal_pred = sample.to('cuda')-out.to('cuda')
            signal_pred_high = torch.where(signal_target.to('cuda').abs()>q.to('cuda')[:, None, None], 
                                       signal_pred.to('cuda'), signal_pred.to('cuda')*stable_scalar)
            
            loss = loss_fn(signal_pred_high.to('cuda'), signal_target_high.to('cuda')) #Loss/error
            val_temp_loss.append(loss.item())
    
    val_loss.append(np.mean(val_temp_loss))
    print(f'validation loss: {np.mean(val_temp_loss)}')
    
    if (epoch+1)%5 == 0: 
        # save model
        torch.save(model.state_dict(), f'noisemodelv4.3_{epoch+1}epochs')


starting epoch 0
training loss: 0.11982080505127234
validation loss: 0.11926767666837279

starting epoch 1
training loss: 0.11135180603271715
validation loss: 0.11405609532843121

starting epoch 2
training loss: 0.11161677636599404
validation loss: 0.11145757083746084

starting epoch 3
training loss: 0.10942747268544115
validation loss: 0.10778775227916353

starting epoch 4
training loss: 0.10945549892652222
validation loss: 0.108294211708429

starting epoch 5
training loss: 0.1086692194083703
validation loss: 0.11464484665048759

starting epoch 6
training loss: 0.10618867461015596
validation loss: 0.11329941164501046

starting epoch 7
training loss: 0.1072774933552776
validation loss: 0.11335938119224574

starting epoch 8
training loss: 0.10669244917776548
validation loss: 0.10714403757040647

starting epoch 9
training loss: 0.10768131626910728
validation loss: 0.11185458278121697

starting epoch 10
training loss: 0.1040926513165108
validation loss: 0.10676630560861804

starting epoc

## Examine results

In [ ]:
random.seed(10)

In [ ]:
#plot loss over all epochs
f, ax = plt.subplots(figsize=(10,5))
ax.plot(train_loss, label='training')
ax.plot(val_loss, label='validation')
ax.set_xlabel('epoch')
ax.set_ylabel('L1 loss')
ax.set_title('Loss')
ax.legend()

In [ ]:
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=1, shuffle=True) #change batch size

num_images = 5
quantile = 0.90

for i, (sample, signal_target, noise_target, era5_noise, dem, era5_corr, hp_corr) in enumerate(val_loader):
    if i < num_images:
        with torch.no_grad():
            noise = model(sample.to('cuda'), dem.to('cuda')) #Generate predictions using the model
            signal = torch.clamp(sample.to('cpu') - noise.to('cpu'), -1, 1)
            q = signal_target.to('cuda').abs().flatten(1, 3).quantile(dim=1, q=quantile, keepdim=True)
            signal_target_high = signal_target.to('cuda').abs()>q.to('cuda')[:, None, None]
            
            
            f, ax = plt.subplots(2, 3, figsize=(15,10))
            ax[0][0].imshow(sample.squeeze(), cmap='RdBu', vmin=-2, vmax=2) 
            ax[0][0].set_title('original interferogram')
            ax[0][1].imshow(noise.squeeze().to('cpu'), cmap='RdBu', vmin=-2, vmax=2)
            ax[0][1].set_title('predicted noise')
            ax[0][2].imshow((sample.squeeze()-signal_target.squeeze()), cmap='RdBu', vmin=-2, vmax=2)
            ax[0][2].set_title('true noise')
            ax[1][1].imshow(signal.squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[1][1].set_title('predicted signal')
            ax[1][2].imshow(signal_target.squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[1][2].set_title('true signal')
            ax[1][0].imshow(signal_target_high.squeeze().to('cpu'))
            ax[1][0].set_title('high loss areas')
            plt.tight_layout()

## Correction comparison

In [ ]:
val_data_ssim = dataset(val_list, val_signal_dir, val_noise_dir, val_dem_dir, val_era5_dir, transform=my_transforms, 
                        blurnoise=True)
train_data_ssim = dataset(train_list, train_signal_dir, train_noise_dir, train_dem_dir, train_era5_dir, transform=my_transforms, 
                        blurnoise=True)
val_loader_ssim = torch.utils.data.DataLoader(dataset = val_data_ssim, batch_size=1, shuffle=False)
train_loader_ssim = torch.utils.data.DataLoader(dataset = train_data_ssim, batch_size=1, shuffle=False)

In [ ]:
# examine corrections
num_images = 5

for i, (sample, signal_target, noise_target, era5_noise, dem, era5_corr, hp_corr) in enumerate(val_loader_ssim):
    if i < num_images:
            noise_pred = model(sample.to('cuda'), dem.to('cuda')) #Generate predictions using the model
            signal_pred = torch.clamp(sample.to('cpu') - noise_pred.to('cpu'), -1, 1)
            f, ax = plt.subplots(2, 3, figsize=(10,7))
            ax[0, 0].imshow(sample.squeeze(), cmap='RdBu', vmin=-2, vmax=2) 
            ax[0, 0].set_title('training')
            ax[0, 1].imshow(signal_target.squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[0, 1].set_title('target signal')
            ax[0, 2].imshow((sample.squeeze()-signal_target.squeeze()), cmap='RdBu', vmin=-2, vmax=2)
            ax[0, 2].set_title('target noise')
            
            ax[1, 0].imshow(signal_pred.detach().squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[1, 0].set_title('model corrected')
            ax[1, 1].imshow(era5_corr.squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[1, 1].set_title('ERA5 corrected')
            ax[1, 2].imshow(hp_corr.squeeze(), cmap='RdBu', vmin=-2, vmax=2)
            ax[1, 2].set_title('hp filter corrected')
            f.tight_layout()

In [ ]:
def ssim_lists(model, data_loader):
    # Calculate val SSIM 
    ssim_list_uncorrected = []
    ssim_list_model_corrected = []
    ssim_list_era5_corrected = []
    ssim_list_hp_corrected = []
    
    for i, (sample, signal_target, noise_target, era5_noise, dem, era5_corr, hp_corr) in enumerate(data_loader):
        # uncorrected SSIM
        ssim_value_uncorrected = ssim(sample.squeeze().detach().numpy(), signal_target.squeeze().detach().numpy(),
                         gaussian_weights=True)
        ssim_list_uncorrected.append(ssim_value_uncorrected)
    
        # model corrected SSIM
        noise = model(sample.to('cuda'), dem.to('cuda')) #Generate predictions using the model
        signal = torch.clamp(sample.to('cpu') - noise.to('cpu'), -1, 1)
        ssim_value_model_corrected = ssim(signal.squeeze().detach().numpy(), signal_target.squeeze().detach().numpy(),
                         gaussian_weights=True)
        ssim_list_model_corrected.append(ssim_value_model_corrected)
    
        # era5 corrected SSIM
        ssim_value_era5_corrected = ssim(era5_corr.squeeze().numpy(), signal_target.squeeze().numpy(),
                         gaussian_weights=True)
        ssim_list_era5_corrected.append(ssim_value_era5_corrected)
    
        # hp filter corrected SSIM
        ssim_value_hp_corrected = ssim(hp_corr.squeeze().numpy(), signal_target.squeeze().numpy(),
                         gaussian_weights=True)
        ssim_list_hp_corrected.append(ssim_value_hp_corrected)
    
    
    print('mean ssim before correction:', np.mean(ssim_list_uncorrected),
          '\nmean ssim model correction:', np.mean(ssim_list_model_corrected), 
          '\nmean ssim era5 correction:', np.mean(ssim_list_era5_corrected),
          '\nmean ssim high pass filter correction:', np.mean(ssim_list_hp_corrected))
    
    return ssim_list_uncorrected, ssim_list_model_corrected, ssim_list_era5_corrected, ssim_list_hp_corrected

In [ ]:
print('val data ssim')
val_ssim_list_uncorrected, val_ssim_list_model, val_ssim_list_era5, val_ssim_list_hp = ssim_lists(model, val_loader_ssim)
print('training data ssim')
train_ssim_list_uncorrected, train_ssim_list_model, train_ssim_list_era5, train_ssim_list_hp = ssim_lists(model, train_loader_ssim)

In [ ]:
# Calculate SNR
def rms(tensor):
    rms = np.sqrt(np.mean(tensor.squeeze().numpy()**2))
    return rms

def snr(model, data_loader):
    snr_list = []

    for i, (sample, signal_target, noise_target, era5_noise, dem, era5_corr, hp_corr) in enumerate(data_loader):
        snr_list.append(rms(signal_target)/rms(sample-signal_target))

    print('mean snr of images:', np.mean(snr_list))
    
    return snr_list

In [ ]:
val_snr_list = snr(model, val_loader_ssim)
train_snr_list = snr(model, train_loader_ssim)

In [ ]:
def df_for_plotting(snr_list, ssim_list_uncorrected, ssim_list_model, ssim_list_era5, ssim_list_hp):

    roll_count = 200
    q_low = 25
    q_high = 75

    ssim_dict = {'snr': snr_list,
                     'ssim_uncorrected':ssim_list_uncorrected,
                     'ssim_model':ssim_list_model,
                     'ssim_era5':ssim_list_era5, 
                     'ssim_hp':ssim_list_hp}
    ssim_df = pd.DataFrame(ssim_dict)

    # uncorrected ssim
    ssim_df['ssim_uncorrected_median'] = ssim_df.sort_values(by=['snr']).ssim_uncorrected.rolling(roll_count, center=True).median()
    ssim_df[f'ssim_uncorrected_q{q_low}'] = ssim_df.sort_values(by=['snr']).ssim_uncorrected.rolling(roll_count, center=True).quantile(quantile=q_low/100)
    ssim_df[f'ssim_uncorrected_q{q_high}'] = ssim_df.sort_values(by=['snr']).ssim_uncorrected.rolling(roll_count, center=True).quantile(quantile=q_high/100)

    # model corrected ssim
    ssim_df['ssim_model_median'] = ssim_df.sort_values(by=['snr']).ssim_model.rolling(roll_count, center=True).median()
    ssim_df[f'ssim_model_q{q_low}'] = ssim_df.sort_values(by=['snr']).ssim_model.rolling(roll_count, center=True).quantile(quantile=q_low/100)
    ssim_df[f'ssim_model_q{q_high}'] = ssim_df.sort_values(by=['snr']).ssim_model.rolling(roll_count, center=True).quantile(quantile=q_high/100)

    # era5 corrected ssim
    ssim_df['ssim_era5_median'] = ssim_df.sort_values(by=['snr']).ssim_era5.rolling(roll_count, center=True).median()
    ssim_df[f'ssim_era5_q{q_low}'] = ssim_df.sort_values(by=['snr']).ssim_era5.rolling(roll_count, center=True).quantile(quantile=q_low/100)
    ssim_df[f'ssim_era5_q{q_high}'] = ssim_df.sort_values(by=['snr']).ssim_era5.rolling(roll_count, center=True).quantile(quantile=q_high/100)

    # era5 corrected ssim
    ssim_df['ssim_hp_median'] = ssim_df.sort_values(by=['snr']).ssim_hp.rolling(roll_count, center=True).median()
    ssim_df[f'ssim_hp_q{q_low}'] = ssim_df.sort_values(by=['snr']).ssim_hp.rolling(roll_count, center=True).quantile(quantile=q_low/100)
    ssim_df[f'ssim_hp_q{q_high}'] = ssim_df.sort_values(by=['snr']).ssim_hp.rolling(roll_count, center=True).quantile(quantile=q_high/100)
    
    return ssim_df

In [ ]:
val_ssim_df=df_for_plotting(val_snr_list, val_ssim_list_uncorrected, val_ssim_list_model, val_ssim_list_era5, val_ssim_list_hp)
train_ssim_df=df_for_plotting(train_snr_list, train_ssim_list_uncorrected, train_ssim_list_model, train_ssim_list_era5, train_ssim_list_hp)

In [ ]:
sns.set_theme()

f, ax = plt.subplots(4, 2, figsize=(10,10), sharex=True, sharey=True)

# val uncorrected 
sns.histplot(ax=ax[0, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_uncorrected, cmap='Oranges', cbar=False, alpha=0)
ax[0, 0].set_xscale('log')

sns.histplot(ax=ax[0, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_uncorrected, cmap='Oranges', cbar=True,
             bins=30, vmax=50, alpha=0.8)
ax[0, 0].set_xscale('log')
ax[0, 0].set_ylabel('SSIM')
ax[0, 0].set_title('uncorrected images (validation)')

sns.lineplot(ax=ax[0, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_uncorrected_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[0, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_uncorrected_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[0, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_uncorrected_q75, size=1, c='gray', legend=False, alpha=0.6)

# val model corrected
sns.histplot(ax=ax[1, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_model, cmap='Oranges', cbar=True,
             bins=30, vmax=50, alpha=0.8)
ax[1, 0].set_xscale('log')
ax[1, 0].set_ylabel('SSIM')
ax[1, 0].set_title('model correction (validation)')

sns.lineplot(ax=ax[1, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_model_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[1, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_model_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[1, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_model_q75, size=1, c='gray', legend=False, alpha=0.6)

# val era5 corrected
sns.histplot(ax=ax[2, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_era5, cmap='Oranges', cbar=True,
             bins=30, vmax=50, alpha=0.8)
ax[2, 0].set_xscale('log')
ax[2, 0].set_ylabel('SSIM')
ax[2, 0].set_title('ERA5 correction (validation)')

sns.lineplot(ax=ax[2, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_era5_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[2, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_era5_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[2, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_era5_q75, size=1, c='gray', legend=False, alpha=0.6)

# val hp corrected
sns.histplot(ax=ax[3, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_hp, cmap='Oranges', cbar=True,
             bins=30, vmax=50, alpha=0.8)
ax[3, 0].set_xscale('log')
ax[3, 0].set_xlabel('SNR (signal/atmosphere)')
ax[3, 0].set_ylabel('SSIM')
ax[3, 0].set_title('high-pass filter correction (validation)')

sns.lineplot(ax=ax[3, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_hp_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[3, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_hp_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[3, 0], x=val_ssim_df.snr, y=val_ssim_df.ssim_hp_q75, size=1, c='gray', legend=False, alpha=0.6)

# train uncorrected 
sns.histplot(ax=ax[0, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_uncorrected, cmap='Blues', cbar=True, 
             bins=30, vmax=100, alpha=0.8)
ax[0, 1].set_xscale('log')
ax[0, 1].set_title('uncorrected images (training)')

sns.lineplot(ax=ax[0, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_uncorrected_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[0, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_uncorrected_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[0, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_uncorrected_q75, size=1, c='gray', legend=False, alpha=0.6)

# train model corrected
sns.histplot(ax=ax[1, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_model, cmap='Blues', cbar=True, 
             bins=30, vmax=100, alpha=0.8)
ax[1, 1].set_xscale('log')
ax[1, 1].set_title('model correction (training)')
f.tight_layout()

sns.lineplot(ax=ax[1, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_model_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[1, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_model_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[1, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_model_q75, size=1, c='gray', legend=False, alpha=0.6)

# train era5 corrected
sns.histplot(ax=ax[2, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_era5, cmap='Blues', cbar=True, 
             bins=30, vmax=100, alpha=0.8)
ax[2, 1].set_xscale('log')
ax[2, 1].set_title('ERA5 correction (training)')
f.tight_layout()

sns.lineplot(ax=ax[2, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_era5_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[2, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_era5_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[2, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_era5_q75, size=1, c='gray', legend=False, alpha=0.6)

# train hp corrected
sns.histplot(ax=ax[3, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_hp, cmap='Blues', cbar=True, 
             bins=30, vmax=100, alpha=0.8)
ax[3, 1].set_xscale('log')
ax[3, 1].set_xlabel('SNR (signal/atmosphere)')
ax[3, 1].set_title('high-pass filter correction (training)')
f.tight_layout()

sns.lineplot(ax=ax[3, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_hp_median, size=1, c='k', legend=False, alpha=0.6)
sns.lineplot(ax=ax[3, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_hp_q25, size=1, c='gray', legend=False, alpha=0.6)
sns.lineplot(ax=ax[3, 1], x=train_ssim_df.snr, y=train_ssim_df.ssim_hp_q75, size=1, c='gray', legend=False, alpha=0.6)


plt.savefig('SSIMv1.png', dpi=300)

In [ ]:
# violin plots 
ssim_labels = ['ssim_uncorrected',  'ssim_model', 'ssim_era5', 'ssim_hp']
val_ssim_long = pd.melt(val_ssim_df[ssim_labels], value_vars=ssim_labels, var_name='corr_type', value_name='ssim')
val_ssim_long['dataset'] = 'val'

ssim_labels = ['ssim_uncorrected',  'ssim_model', 'ssim_era5', 'ssim_hp']
train_ssim_long = pd.melt(train_ssim_df[ssim_labels], value_vars=ssim_labels, var_name='corr_type', value_name='ssim')
train_ssim_long['dataset'] = 'train'

all_ssim_long = pd.concat([train_ssim_long, val_ssim_long])

In [ ]:
f, ax = plt.subplots()
sns.violinplot(ax=ax, data=all_ssim_long, x="corr_type", y="ssim", hue='dataset')
ax.set_xticklabels(['uncorrected', 'CNN', 'ERA5', 'gaussian filter'])
ax.set_xlabel('correction type')
ax.set_ylabel('SSIM')
plt.savefig('SSIM_violin.png', dpi=300)

In [ ]:
# test if data are normally distributed
print('val uncorrected: ', stats.kstest(val_ssim_df['ssim_uncorrected'].values, stats.norm.cdf))
print('val model: ', stats.kstest(val_ssim_df['ssim_model_corrected'].values, stats.norm.cdf))
print('val era5: ', stats.kstest(val_ssim_df['ssim_era5_corrected'].values, stats.norm.cdf))
print('val hp: ', stats.kstest(val_ssim_df['ssim_hp_corrected'].values, stats.norm.cdf))

#they are not even close to normal

In [ ]:
# wilcoxon signed rank test to compare paired samples
stats.wilcoxon(val_ssim_df['ssim_uncorrected'].values, val_ssim_df['ssim_model_corrected'].values, nan_policy='omit')

In [ ]:
stats.wilcoxon(val_ssim_df['ssim_model_corrected'].values, val_ssim_df['ssim_era5_corrected'].values, nan_policy='omit')

In [ ]:
stats.wilcoxon(val_ssim_df['ssim_model_corrected'].values, val_ssim_df['ssim_hp_corrected'].values, nan_policy='omit')

## Visualize feature maps

In [ ]:
# visualize feature maps

activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

cnn_names = ['cnn1', 'cnn2', 'cnn3','cnn4','cnn5', 'cnn6', 'cnn7', 'cnn8'] 
cnn_vars = [model.cnn1, model.cnn2, model.cnn3, model.cnn4, model.cnn5,
            model.cnn6, model.cnn7, model.cnn8]


def plot_feature(cnn_vars, cnn_names, num_images=1):
    for i, (sample, signal_target, noise_target, dem) in enumerate(val_loader):
        if i<num_images:
            for j, cnn_name in enumerate(cnn_names):
                cnn = cnn_vars[j]
                cnn.register_forward_hook(get_activation(cnn_name))
                output = model(sample.to('cuda'), dem.to('cuda'))

                act = activation[cnn_name].squeeze()
                act1 = act[0:15,:,:].sum(dim=0, keepdim=True).squeeze()
                act2 = act[16:31,:,:].sum(dim=0, keepdim=True).squeeze()
                act3 = act[32:47,:,:].sum(dim=0, keepdim=True).squeeze()
                act4 = act[48:63,:,:].sum(dim=0, keepdim=True).squeeze()

                fig, ax = plt.subplots(1, 5, figsize=(19,5))
                #ax[0].imshow(dem.squeeze().to('cpu'), cmap='Greys')
                ax[0].imshow(sample.squeeze().to('cpu'), cmap='RdBu')
                ax[0].set_title('original image')
                ax[1].imshow(act1.to('cpu'))
                ax[1].set_title(f'{cnn_name} feature map')
                ax[2].imshow(act2.to('cpu'))
                ax[2].set_title(f'{cnn_name} feature map')
                ax[3].imshow(act3.to('cpu'))
                ax[3].set_title(f'{cnn_name} feature map')
                ax[4].imshow(act4.to('cpu'))
                ax[4].set_title(f'{cnn_name} feature map')
        else:
            break
            
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=1, shuffle=True)

In [ ]:
plot_feature(cnn_vars, cnn_names, num_images=1)